In [ ]:
%pip install paddlepaddle

In [ ]:
%pip install paddleocr


In [ ]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from google.colab.patches import cv2_imshow

In [ ]:
ocr = PaddleOCR(lang='en')

# Path to your recorded video
video_path = '/content/Untitled video - Made with Clipchamp.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Unable to open the video file.")
    exit()

# Create a VideoWriter object to save the output video
output_path = '/content/output_video.mp4'
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 25, (frame_width, frame_height))

# Loop through each frame in the video
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame is valid
    if not ret:
        break

    # Perform OCR on the frame
    result = ocr.ocr(frame)

    # Check if OCR result is None (no text detected)
    if result is None:
        continue  # Skip processing if no text is detected

    # Draw bounding boxes and text on the frame
    image_with_text = frame.copy()  # Create a copy of the frame
    for line in result:
        if line is None:
            continue
        for word_info in line:
            bbox = word_info[0]
            text = word_info[1][0]  # Extract the recognized text
            # Draw bounding box
            bbox = np.array(bbox).reshape((-1, 1, 2)).astype(np.int32)
            cv2.polylines(image_with_text, [bbox], isClosed=True, color=(0, 255, 0), thickness=2)
            # Draw text
            cv2.putText(image_with_text, text, (bbox[0][0][0], bbox[0][0][1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Write the frame with drawn bounding boxes and text to the output video
    out.write(image_with_text)

    # Display the frame with bounding boxes and text using cv2_imshow
    cv2_imshow(image_with_text)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object, release the video writer, and close all windows
cap.release()
out.release()

print("License plate recognition completed. Output video saved as:", output_path)
